In [ ]:
!git clone https://github.com/pangdatangtt/UniNet.git

Cloning into 'UniNet'...
remote: Enumerating objects: 373, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 373 (delta 104), reused 66 (delta 66), pack-reused 255 (from 2)
Receiving objects: 100% (373/373), 6.63 MiB | 24.86 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [ ]:
# Önce varsa eski linki sil (opsiyonel)
!rm -rf /content/UniNet/data

# Yeni bağlantıyı oluştur
!ln -s "/content/drive/MyDrive/Wood_Dataset" "/content/UniNet/data"




In [ ]:
!python "/content/UniNet/main.py" --dataset "MVTec AD" --setting oc --save_dir "./results" --epoch 50


training on MVTec AD dataset (separate-class)
class:wood, domain:industrial, setting:oc, dataset:MVTec AD, epochs:50, batch_size:8, image_size:256, center_crop:256, lr_s:0.005, lr_t:1e-06, T:2, weighted_decision_mechanism:True, default:0.3, alpha:0.01, beta:3e-05, train_and_test_all:True, is_saved:True, save_dir:./results, load_ckpts:False, 
cuda
epoch [1/50], loss:20.4138
epoch [2/50], loss:19.7259
epoch [3/50], loss:19.5118
epoch [4/50], loss:19.3981
epoch [5/50], loss:19.3338
epoch [6/50], loss:19.2665
epoch [7/50], loss:19.2263
epoch [8/50], loss:19.1874
epoch [9/50], loss:19.1668
epoch [10/50], loss:19.1332
fps: 12.912074008191242 141
Sample Auroc: 55.3, Pixel Auroc: 78.1, Pixel Aupro: 32.9
modules have been saved to ./ckpts/MVTec AD/wood/BEST_P_ROC.pth
saved
modules have been saved to ./ckpts/MVTec AD/wood/BEST_P_PRO.pth
MAX I_ROC: 55.3, MAX P_ROC: 78.1, MAX P_PRO: 32.9
epoch [11/50], loss:19.1033
epoch [12/50], loss:19.0833
epoch [13/50], loss:19.0590
epoch [14/50], loss:19.0440

In [ ]:
!python '/content/UniNet/main.py' --dataset "MVTec AD" --setting oc --save_dir "./results" --load_ckpts

training on MVTec AD dataset (separate-class)
class:wood, domain:industrial, setting:oc, dataset:MVTec AD, epochs:100, batch_size:8, image_size:256, center_crop:256, lr_s:0.005, lr_t:1e-06, T:2, weighted_decision_mechanism:True, default:0.3, alpha:0.01, beta:3e-05, train_and_test_all:True, is_saved:True, save_dir:./results, load_ckpts:False, 
cuda
epoch [1/100], loss:20.4138
epoch [2/100], loss:19.7259
epoch [3/100], loss:19.5118
epoch [4/100], loss:19.3981
epoch [5/100], loss:19.3338
epoch [6/100], loss:19.2665
epoch [7/100], loss:19.2263
epoch [8/100], loss:19.1874
epoch [9/100], loss:19.1668
epoch [10/100], loss:19.1332
fps: 12.959189228152601 141
Sample Auroc: 55.3, Pixel Auroc: 78.1, Pixel Aupro: 32.9
modules have been saved to ./ckpts/MVTec AD/wood/BEST_P_ROC.pth
saved
modules have been saved to ./ckpts/MVTec AD/wood/BEST_P_PRO.pth
MAX I_ROC: 55.3, MAX P_ROC: 78.1, MAX P_PRO: 32.9
epoch [11/100], loss:19.1033
epoch [12/100], loss:19.0833
epoch [13/100], loss:19.0590
epoch [14/100

In [59]:
def test(c, suffix='BEST_P_PRO'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🖥️  Device: {device}")

    dataset_name = c.dataset

    # 🔁 SENİN DOĞRUDAN VERDİĞİN CHECKPOINT PATH
    ckpt_path = "/content/UniNet/ckpts/ckpts/MVTec AD/wood"  # ← burada BEST_P_PRO.pth var

    # Dataset yükle
    dataset_info = loading_dataset(c, dataset_name)
    test_dataloader = dataset_info[1]

    # Model tanımı ve ağırlık yükleme
    Source_teacher, bn = wide_resnet50_2(c, pretrained=True)
    Source_teacher.layer4 = None
    Source_teacher.fc = None
    student = de_wide_resnet50_2(pretrained=False)
    DFS = DomainRelated_Feature_Selection()
    [Source_teacher, bn, student, DFS] = to_device([Source_teacher, bn, student, DFS], device)
    Target_teacher = copy.deepcopy(Source_teacher)

    new_state = load_weights([Target_teacher, bn, student, DFS], ckpt_path, suffix)
    Target_teacher = new_state['tt']
    bn = new_state['bn']
    student = new_state['st']
    DFS = new_state['dfs']

    model = UniNet(c, Source_teacher.cuda().eval(), Target_teacher, bn, student, DFS)

    # Performans metriği hesapla
    auroc_px, auroc_sp, pro = evaluation_indusAD(c, model, test_dataloader, device)

    return auroc_sp, auroc_px, pro, model, device


In [ ]:
def test_all_images(c, model, device):
    import glob
    import os
    from PIL import Image
    import numpy as np
    import matplotlib.pyplot as plt
    import cv2
    import datetime
    from torchvision import transforms as T
    from scipy.ndimage import gaussian_filter
    import torch
    from torch.nn import functional as F

    test_images_dir = f"/content/UniNet/data/mvtec/{c._class_}/test/defect/"
    gt_dir = f"/content/UniNet/data/mvtec/{c._class_}/ground_truth/defect/"

    image_paths = sorted(glob.glob(os.path.join(test_images_dir, "*.jpg")))
    os.makedirs("all_results", exist_ok=True)

    for img_path in image_paths:
        filename = os.path.basename(img_path)
        gt_path = os.path.join(gt_dir, filename.replace(".jpg", "_mask.jpg"))

        original_image = Image.open(img_path).convert('RGB')
        transform = T.Compose([
            T.Resize((c.image_size, c.image_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = transform(original_image).unsqueeze(0).to(device)

        model.train_or_eval(type='eval')
        n = model.n
        output_list = [list() for _ in range(n * 3)]

        with torch.no_grad():
            t_tf, de_features = model(input_tensor)
            for l, (t, s) in enumerate(zip(t_tf, de_features)):
                output = 1 - F.cosine_similarity(t, s)
                output_list[l].append(output)

            anomaly_score, anomaly_map = weighted_decision_mechanism(1, output_list, c.alpha, c.beta)
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            anomaly_map = anomaly_map.squeeze()

        # Normalize heatmap
        anomaly_map_norm = cv2.normalize(anomaly_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        heatmap = cv2.applyColorMap(anomaly_map_norm, cv2.COLORMAP_JET)

        # Contours
        _, thresh = cv2.threshold(anomaly_map_norm, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        original_cv = np.array(original_image.resize((c.image_size, c.image_size)))
        original_cv = cv2.cvtColor(original_cv, cv2.COLOR_RGB2BGR)
        contour_image = cv2.drawContours(original_cv.copy(), contours, -1, (0, 255, 0), 2)

        # GT mask
        if os.path.exists(gt_path):
            gt_mask = Image.open(gt_path).convert('L').resize((c.image_size, c.image_size))
        else:
            gt_mask = np.zeros_like(anomaly_map_norm)

        # Görseli Kaydet
        label = "defect" if "defect" in img_path.lower() else "good"
        img_number = filename.replace(".jpg", "")
        save_path = f"all_result/result_{label}_{img_number}.png"

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 4, 1); plt.title("Original"); plt.imshow(original_image.resize((c.image_size, c.image_size))); plt.axis('off')
        plt.subplot(1, 4, 2); plt.title("Anomaly Map"); plt.imshow(anomaly_map, cmap='jet'); plt.axis('off')
        plt.subplot(1, 4, 3); plt.title("Contours"); plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)); plt.axis('off')
        plt.subplot(1, 4, 4); plt.title("Ground Truth"); plt.imshow(gt_mask, cmap='gray'); plt.axis('off')
        plt.savefig(save_path)
        plt.close()

        print(f"[✓] {filename} -> Max Score: {anomaly_map.max().item():.4f}, saved to {save_path}")


In [ ]:
auroc_sp, auroc_px, pro, model, device = test(c)
test_all_images(c, model, device)

🖥️  Device: cuda
Loading weights from /content/ckpts/MVTec AD/wood/BEST_P_PRO.pth
fps: 12.88794413602297 141
[✓] 100000000.jpg -> Max Score: 2.1927, saved to all_result/result_defect_100000000.png
[✓] 100000003.jpg -> Max Score: 2.2528, saved to all_result/result_defect_100000003.png
[✓] 100000006.jpg -> Max Score: 2.3348, saved to all_result/result_defect_100000006.png
[✓] 100000007.jpg -> Max Score: 2.4950, saved to all_result/result_defect_100000007.png
[✓] 100000008.jpg -> Max Score: 2.2454, saved to all_result/result_defect_100000008.png
[✓] 100000009.jpg -> Max Score: 2.1623, saved to all_result/result_defect_100000009.png
[✓] 100000012.jpg -> Max Score: 2.3420, saved to all_result/result_defect_100000012.png
[✓] 100000014.jpg -> Max Score: 2.4141, saved to all_result/result_defect_100000014.png
[✓] 100000016.jpg -> Max Score: 2.1043, saved to all_result/result_defect_100000016.png
[✓] 100000018.jpg -> Max Score: 2.1655, saved to all_result/result_defect_100000018.png
[✓] 1000000

In [75]:
def test_all_images_gd(c, model, device):
    import glob
    import os
    from PIL import Image
    import numpy as np
    import matplotlib.pyplot as plt
    import cv2
    import datetime
    from torchvision import transforms as T
    from scipy.ndimage import gaussian_filter
    import torch
    from torch.nn import functional as F

    test_images_dir = f"/content/UniNet/data/mvtec/{c._class_}/test/*/"
    gt_dir = f"/content/UniNet/data/mvtec/{c._class_}/ground_truth/"

    image_paths = sorted(glob.glob(os.path.join(test_images_dir, "*.jpg")))
    os.makedirs("all_results", exist_ok=True)

    print(f"Toplam {len(image_paths)} test görüntüsü bulundu.")

    # 1️⃣ Tüm anomaly map max skorlarını topla
    max_scores = []
    anomaly_maps = []
    processed = []

    for img_path in image_paths:
        filename = os.path.basename(img_path)

        original_image = Image.open(img_path).convert('RGB')
        transform = T.Compose([
            T.Resize((c.image_size, c.image_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = transform(original_image).unsqueeze(0).to(device)

        model.train_or_eval(type='eval')
        n = model.n
        output_list = [list() for _ in range(n * 3)]

        with torch.no_grad():
            t_tf, de_features = model(input_tensor)
            for l, (t, s) in enumerate(zip(t_tf, de_features)):
                output = 1 - F.cosine_similarity(t, s)
                output_list[l].append(output)

            anomaly_score, anomaly_map = weighted_decision_mechanism(1, output_list, c.alpha, c.beta)
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            anomaly_map = anomaly_map.squeeze()

        max_score = anomaly_map.max().item()
        label = "defect" if max_score >= 1.98 else "good"

        max_scores.append(max_score)
        anomaly_maps.append(anomaly_map)
        processed.append((img_path, original_image, anomaly_map, label, max_score))

    global_max = max(max_scores)
    print(f"\n🌡️ Global max anomaly score: {global_max:.4f}\n")

    # 2️⃣ Normalize edip görselleri kaydet
    for idx, (img_path, original_image, anomaly_map, label, max_score) in enumerate(processed):
        filename = os.path.basename(img_path)
        gt_path = os.path.join(gt_dir, label, filename.replace(".jpg", "_mask.jpg"))

        anomaly_map_norm = (anomaly_map / global_max * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(anomaly_map_norm, cv2.COLORMAP_JET)

        _, thresh = cv2.threshold(anomaly_map_norm, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        original_cv = np.array(original_image.resize((c.image_size, c.image_size)))
        original_cv = cv2.cvtColor(original_cv, cv2.COLOR_RGB2BGR)
        contour_image = cv2.drawContours(original_cv.copy(), contours, -1, (0, 255, 0), 2)

        # GT mask
        if os.path.exists(gt_path):
            gt_mask = Image.open(gt_path).convert('L').resize((c.image_size, c.image_size))
        else:
            gt_mask = np.zeros_like(anomaly_map_norm)

        save_path = f"all_results/result_{label}_{filename.replace('.jpg', '')}.png"
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 4, 1); plt.title("Original"); plt.imshow(original_image.resize((c.image_size, c.image_size))); plt.axis('off')
        plt.subplot(1, 4, 2); plt.title("Anomaly Map"); plt.imshow(anomaly_map, cmap='jet'); plt.axis('off')
        plt.subplot(1, 4, 3); plt.title("Contours"); plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)); plt.axis('off')
        plt.subplot(1, 4, 4); plt.title("Ground Truth"); plt.imshow(gt_mask, cmap='gray'); plt.axis('off')
        plt.savefig(save_path)
        plt.close()

        print(f"[{idx+1}/{len(processed)}] ✓ {filename} → Max Score: {max_score:.4f}, Predicted: {label}, Saved to: {save_path}")

auroc_sp, auroc_px, pro, model, device = test(c)
test_all_images_gd(c, model, device)


🖥️  Device: cuda
Loading weights from /content/UniNet/ckpts/ckpts/MVTec AD/wood/BEST_P_PRO.pth
fps: 12.813016603300685 141
Toplam 141 test görüntüsü bulundu.

🌡️ Global max anomaly score: 2.7611

[1/141] ✓ 100000000.jpg → Max Score: 2.1927, Predicted: defect, Saved to: all_results/result_defect_100000000.png
[2/141] ✓ 100000003.jpg → Max Score: 2.2528, Predicted: defect, Saved to: all_results/result_defect_100000003.png
[3/141] ✓ 100000006.jpg → Max Score: 2.3348, Predicted: defect, Saved to: all_results/result_defect_100000006.png
[4/141] ✓ 100000007.jpg → Max Score: 2.4950, Predicted: defect, Saved to: all_results/result_defect_100000007.png
[5/141] ✓ 100000008.jpg → Max Score: 2.2454, Predicted: defect, Saved to: all_results/result_defect_100000008.png
[6/141] ✓ 100000009.jpg → Max Score: 2.1623, Predicted: defect, Saved to: all_results/result_defect_100000009.png
[7/141] ✓ 100000012.jpg → Max Score: 2.3420, Predicted: defect, Saved to: all_results/result_defect_100000012.png
[8/141

In [74]:
!rm -rf /content/UniNet/all_results